# Initiailize VW executor

In [ ]:
from vw_pipeline.handlers import WidgetHandler
from vw_pipeline.vw_cache import VwCache
from vw_pipeline.vw import Vw
from vw_pipeline.vw_opts import dimension, product
import pandas as pd

#your vw path
vw_path = r'vw'

cache = VwCache(r'_cache1')
vw = Vw(vw_path,
    cache,
    handlers=[WidgetHandler()],
 #   loggers=[Loggers.ConsoleLogger('DEBUG')]
    )

# Get the data

In [ ]:
inputs = [
    #'file1.json',
    #'file2.json',
    ]
inputs

# Define options grid and train

In [ ]:
opts = product(
    dimension('#base', ['--ccb_explore_adf -P 10000 --preserve_performance_counters --save_resume --dsjson']),
    dimension('--cb_type', ['ips', 'mtr']),
)
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Get access to losses and job artifacts

In [ ]:
print(f'loss[0]: {result[0].loss}')
print(f'outputs: {result[0].outputs}')

# Pandas wrappers

In [ ]:
opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf -P 100 --preserve_performance_counters --save_resume']),
    dimension('--cb_type', ['ips', 'mtr']),
    dimension('#format', ['', '--dsjson'])
))
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Investigate failure

In [ ]:
result[~result['!Loss'].notnull()].iloc[0]['!Job'].tasks[0].stdout()

## Explore metrics 

In [ ]:
result.iloc[1]['!Metrics'].head()

In [ ]:
import matplotlib.pyplot as plt
[(-r['!Metrics']['loss_per_example']).plot(label=f"{r['!Job'].name}", figsize=(10,6)) for i, r in result[result['!Loss'].notnull()].iterrows()]
plt.legend()